# Test MFCC

In [17]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import librosa

sr=1600
audio=np.load("sd_GSCmdV2/train/on/00b01445_nohash_0.wav.npy")
print(audio.shape)

mfccs=librosa.feature.mfcc(y=audio,sr=sr, n_mfcc=40)
print(mfccs.shape)

mean = np.mean(mfccs, axis=1)
std = np.std(mfccs, axis=1)

mfccs=(mfccs-mean)/std

plt.imshow(mfccs)
plt.colorbar()
plt.ylabel('Cepstral Coefficients')
plt.xlabel('Time')
plt.show()

(16000,)
(40, 32)


ValueError: operands could not be broadcast together with shapes (40,32) (40,) 